## Cómo optimizar la consulta de datos en Amazon S3

La consulta de datos de forma eficiente y eficaz se ha convertido en una capacidad crítica para los clientes de todos los sectores. Al consultar los datos, los clientes pueden filtrar su contenido y recuperar sólo el subconjunto de datos que necesitan. Los clientes necesitan la capacidad de consultar y recuperar subconjuntos específicos de datos relevantes para sus necesidades u objetivos actuales. Al extraer información de los datos, las organizaciones pueden obtener información valiosa, tomar decisiones basadas en datos y, en última instancia, impulsar el éxito empresarial en un panorama cada vez más competitivo y basado en datos.

Con los datos en Amazon S3, dispone de varias opciones para consultarlos, incluido el filtrado del lado del cliente, Amazon Athena y Amazon S3 Object Lambda. En primer lugar, puede descargar un objeto de S3 en su instancia informática de AWS y, a continuación, consultarlo. Esta opción proporciona un alto desempeño para consultar un único objeto de S3 o un objeto restaurado desde las clases de almacenamiento de Amazon S3 Glacier. En segundo lugar, para casos de uso de lago de datos en los que desee consultar varios objetos de S3 a la vez, recomendamos utilizar Athena. Athena no tiene servidor, por lo que no hay que configurar ni administrar ninguna infraestructura. En tercer lugar, puede utilizar S3 Object Lambda para añadir su propio código a las API GET, HEAD y LIST de S3 para transformar los datos a medida que se devuelven a sus aplicaciones. Puedes beneficiarte de S3 Object Lambda para casos de uso en los que quieras añadir transformaciones mientras consultas datos.

1. **Filtrado del lado del cliente**. Puede descargar un objeto de S3 en su instancia informática de AWS y consultar el contenido utilizando bibliotecas de análisis de datos en su aplicación cliente. Por ejemplo, en una aplicación Python que consulta objetos Parquet en S3, puede utilizar la API GET de S3 para descargar el objeto y consultar el contenido utilizando la biblioteca 'Pandas'.

    Esta opción proporciona un alto rendimiento entre las opciones disponibles en la mayoría de los casos, dependiendo del ratio de filtrado de la consulta y de las propiedades del objeto (tamaño y ratio de compresión). Esto se debe a que el uso de la API GET de S3 para descargar un objeto comprimido y consultarlo en su aplicación cliente suele ser más rápido que transferir los resultados de la consulta más grandes sin comprimir a través de la red. Para obtener más información sobre la API GET de S3, consulte la documentación de la API GetObject de S3.

2. **Amazon Athena**. Amazon Athena es un servicio de consulta interactivo que facilita el análisis de datos directamente en S3 utilizando SQL estándar. Debería utilizar Athena si desea ejecutar consultas SQL interactivas improvisadas contra datos en S3, sin tener que administrar ninguna infraestructura.

    Athena utiliza un motor SQL distribuido, Trino, para ejecutar consultas sobre objetos almacenados en S3, representados como tablas Hive. Después de configurar una tabla, puede utilizar Athena para consultar sus objetos de S3. Puede procesar varios objetos de S3 en una sola consulta o incluso utilizar operaciones de unión y funciones de ventana para consultar sus objetos de S3. Además, Athena admite formatos de datos como ORC y Apache Avro, y también objetos comprimidos con el algoritmo Zstandard. Para obtener más información, consulta la guía del usuario de Amazon Athena, la publicación del blog Analyzing data in S3 using Amazon Athena y la guía del usuario Querying restored data from Glacier using Athena.

3. **Objeto Lambda de Amazon S3**. Con S3 Object Lambda, puede añadir su propio código a las solicitudes GET de S3 para procesar los datos a medida que se devuelven a su aplicación. Al escribir su propio código, puede incluir su lógica de consulta para filtrar solo el subconjunto de datos que se devolverá a su aplicación. S3 Object Lambda es adecuado si quieres añadir transformaciones a los resultados de tus consultas. Por ejemplo, con S3 Object Lambda, puede comprimir los resultados de su consulta con un algoritmo de su elección mientras se devuelve a su aplicación.

    Cuando envía una solicitud a través de S3 Object Lambda, S3 llama automáticamente a una función de AWS Lambda en su cuenta de AWS para procesar los datos recuperados, devolviendo un resultado transformado a la aplicación. AWS Lambda es un servicio informático sin servidor que ejecuta código definido por el usuario sin necesidad de administrar los recursos informáticos subyacentes. Al utilizar Lambda, puede utilizar una biblioteca de análisis de datos en cualquier tiempo de ejecución de Lambda compatible y utilizar cualquier técnica de consulta de datos sin estar limitado al formato SQL. Para obtener más información, consulte la guía del usuario de S3 Object Lambda.

In [4]:
import pandas as pd
import boto3
from boto3.s3.transfer import TransferConfig
from botocore.exceptions import ClientError

s3 = boto3.client("s3")

In [ ]:
# Restore objects from S3 Glacier (if applicable)
s3.restore_object(
	Bucket='examplebucket',
	Key='archivedobjectkey',
	RestoreRequest={
		'Days': 1,
			'GlacierJobParameters': {
				'Tier': 'Standard',
			},
	},
)

In [11]:
def create_bucket(bucket_name, region=None):
    try:
        if region is None:
            s3.create_bucket(Bucket=bucket_name)
        else:
            location = {'LocationConstraint': region}
            s3.create_bucket(Bucket=bucket_name,
                    CreateBucketConfiguration=location)
    except ClientError as e:
        print(e)
        return False
    else:
        print(f"Bucket {bucket_name} created successfully")
        return bucket_name

In [12]:
bucket_name = create_bucket("test-data-4959-1500", region="eu-west-1")

Bucket test-data-4959-1500 created successfully


In [14]:
# Upload a file to an S3 bucket
file_name = "Flights_1m.parquet"
object_name = "Flights_1m.parquet"

try:
    response = s3.upload_file(file_name, bucket_name, object_name)
except ClientError as e:
    print(f"Error uploading file: {e}")

In [15]:
concurrency_value = 4

# Download the S3 object, setting concurrency based on your instance’s processing capacity.
s3.download_file(bucket_name, file_name, 'output.parquet', Config=TransferConfig(max_concurrency=concurrency_value))

In [20]:
# Filter the S3 object
args = {'filters': [('DISTANCE', '>', 3000)]}
df = pd.read_parquet('output.parquet', engine='pyarrow', **args)
df.head()

,FL_DATE,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE,DEP_TIME,ARR_TIME
0,2006-01-01,-4,26,516,3784,10.016666,15.116667
1,2006-01-02,1,25,505,3784,10.100000,15.100000
2,2006-01-03,-2,13,503,3784,10.050000,14.900000
3,2006-01-04,0,21,510,3784,10.083333,15.033334
4,2006-01-05,-3,4,498,3784,10.033334,14.750000
